In [1]:
from status_update import *

# display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
session = get_session("production")

Connected to Aquarium at http://52.27.43.242/ using pydent version 0.0.35
Logged in as Devin Strickland



### Find all DNA Libraries that have been run in Challenge and Label Operations

In [3]:
se = StatusExplorer(session)
operation_type_name = "Challenge and Label"
operation_type = session.OperationType.find_by_name(operation_type_name)

In [6]:
year = 2019
month = 8

if month < 12:
    next_month = month + 1
    next_year = year
else:
    next_month = 1
    next_year = year + 1
    
pacific = timezone('US/Pacific')

month_start = pacific.localize(datetime(year, month, 1))
month_end = pacific.localize(datetime(next_year, next_month, 1))

In [7]:
query = "operation_type_id = '{}' AND updated_at >= '{}' AND updated_at < '{}'"
query_ops = session.Operation.where(query.format(operation_type.id, month_start, month_end))

msg = "Found {} Operations of type {} between {} and {}."
print(msg.format(len(query_ops), operation_type_name, month_start, month_end))

Found 72 Operations of type Challenge and Label between 2019-08-01 00:00:00-07:00 and 2019-09-01 00:00:00-07:00.


In [8]:
query_op_ids = [o.id for o in query_ops]

output_field_values = session.FieldValue.where({
    "parent_id": query_op_ids, 
    "role": "output", 
    "parent_class": "Operation"
})

print("Found {} output FieldValues for Operations of type {}.".format(len(output_field_values), operation_type_name))

Found 72 output FieldValues for Operations of type Challenge and Label.


In [9]:
sample_type_name = "DNA Library"
sample_ids = [fv.child_sample_id for fv in output_field_values]
sample_type = session.SampleType.find_by_name(sample_type_name)

relevant_samples = session.Sample.where({
    "id": sample_ids,
    "sample_type_id": sample_type.id
})

se.set_samples(relevant_samples)
print("Found {} Samples of type {}.".format(len(se.samples), sample_type_name))

Found 3 Samples of type DNA Library.


In [10]:
sample_ids = [s.id for s in se.samples]

relevant_fvs = []
nested_sample_ids = chunks(sample_ids, 10)

for these_sample_ids in nested_sample_ids:
    fvs = session.FieldValue.where({
        "child_sample_id": these_sample_ids,
        "role": "input", 
        "parent_class": "Operation"
    })
    if fvs: relevant_fvs += fvs

se.set_field_values(relevant_fvs)
print("Found {} input FieldValues for Samples of type {}.".format(len(se.field_values), sample_type_name))

Found 1151 input FieldValues for Samples of type DNA Library.


In [11]:
relevant_op_ids = [fv.parent_id for fv in se.field_values]

nested_op_ids = chunks(relevant_op_ids, 1000)
relevant_ops = []
job_associations = []
plan_associations = []

for these_op_ids in nested_op_ids:
    if these_op_ids:
        ops = session.Operation.find(these_op_ids)
        if ops: relevant_ops += ops
            
        jas = session.JobAssociation.where({"operation_id": these_op_ids})
        if jas: job_associations += jas
            
        pas = session.PlanAssociation.where({"operation_id": these_op_ids})
        if pas: plan_associations += pas

se.set_operations(relevant_ops)
se.set_job_associations(job_associations)
se.set_plan_associations(plan_associations)
print("Found {} relevant Operations.".format(len(se.operations)))

Found 1134 relevant Operations.


In [12]:
print(len(se.plan_associations))
print(len(se.job_associations))

1134
811


In [13]:
plan_ids = [pa.plan_id for pa in se.plan_associations]
plan_ids = list(set(plan_ids))
plans = session.Plan.where({"id": plan_ids})
se.set_plans(plans)

In [15]:
job_ids = [ja.job_id for ja in se.job_associations]
job_ids = list(set(job_ids))

jobs = find_in_batches(session.Job, job_ids, 50)
# jobs = [j for j in jobs if j.state[-1]['operation'] == "complete"]
se.set_jobs(jobs)

Found 30% (50) of 165 records
Found 61% (100) of 165 records
Found 91% (150) of 165 records
Found 100% (165) of 165 records


In [16]:
phases = {    
    "Assay": [
        'Innoculate Yeast Library',
        'Mix Cultures',
        'Dilute Yeast Library',
        'Store Yeast Library Sample',
        'Challenge and Label',
        'Sort Yeast Display Library'
    ],
    "NGS Prep": [
        "Treat With Zymolyase",
        "Yeast Plasmid Extraction",
        "Digest Genomic DNA",
        "Make qPCR Fragment",
        "Make qPCR Fragment WITH PLATES",
        "Run Pre-poured Gel",
        "Extract Gel Slice (NGS)",
        "Purify Gel Slice (NGS)",
        "Dilute to 4nM",
        "Qubit concentration"
    ]
}

colors = {
    "Construction": 'rgb(46, 137, 205)',
    "Assay": 'rgb(198, 47, 105)',
    "NGS Prep": 'rgb(107, 127, 135)',
}

def get_phase(ot_name):
    for phase, ot_names in phases.items():
        if ot_name in ot_names:
            return phase
    return "Construction"

In [17]:
gantt_df = []

for sample in se.samples:
    these_ops = se.operations_for(sample=sample)
    these_plans = se.plans_for(operations=these_ops)
    these_jobs = se.jobs_for(operations=these_ops)
    
    if these_jobs:
        for job in these_jobs:
            if len(job.state) < 5:
                continue
            stats = se.get_stats(job)
            start = stats["start_time"]
            finish = stats["stop_time"]
            if start and finish:
                gantt_df.append(dict(Task=sample.name, Start=start, Finish=finish, Resource=get_phase(stats["ot_name"])))

In [19]:
gantt_df = sorted(gantt_df, key=lambda x: x["Start"], reverse=False)

fig = ff.create_gantt(gantt_df, colors=colors, show_colorbar=True, index_col='Resource', 
                      height=600, width=1600, group_tasks=True, showgrid_x=True, showgrid_y=False)
                      
fig.update_layout(template="plotly_white")
fig.show()